# Crawling Rosstat

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
#import sqlite3
import requests
import re
import numpy as np
from datetime import datetime as dt
from datetime import date, timedelta

from pathlib import Path
import os.path

import time
#import sqlalchemy as sa


In [8]:
lstSourceInn=[]
with open('inn_1.txt', 'r') as fl:
    lstSourceInn=fl.read().split('\n')
    print(*lstSourceInn)
    print(len(lstSourceInn))
    print(lstSourceInn[-1])

7703585780 7736050003 7706107510 7736617998 7707572492 5504036333 2310031475 8602060555 5003021311 7705183980 7706061801 7728308080 7728029110 8608048498 7841011317 5003052454 0274051582 7840471255 4703105075 7708729065 1644003838 6316031581 7813458488 5250043567 8603089934 8401005730 7727547261 7740000076 7703270067 7712040126 7727223267 1106014140 4823006703 7709825967 2437261631 7707049388 7708207809 7713076301 7729588440 7703386329 5905099475 8622000931 7736520080 7709303960 7721632827 3528000597 7814148471 3448017919 5027090498 7708636639 7708004767 5003027352 7813309983 7709258228 5612002469 7710390358 7704218694 5042059767 8904034777 7730095858 3840003450 5036045205 7825429040 7710298176 8604035473 5837047444 7801059506 5902201970 7708709686 7707548740 8904034784 7825706086 4716016979 7710619750 7838008963 2608001153 7826087713 7225003194 5408130693 0266008329 7705035012 8618006063 7708730590 1102024468 8905000428 7703266053 7707016368 7728530221 7724053916 5032260402 7704784450

In [3]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.chrome.options import Options as CHROptions
import selenium.webdriver.chrome.service as service

from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.remote.command import Command

strURL=r'https://websbor.gks.ru/online/#!/gs/statistic-codes'
str_def_dir_dwld=r'/home/egor/Загрузки' #'/home/egor/tmp'
#str_def_dir_dwld='/home/egor/tmp'
#strInn='7708503727'

def get_chromium():
    chrome_path='/usr/lib/chromium-browser/chromedriver'

    #service=service.Service('/usr/bin/chromium-browser')
    #service.start()
    options=CHROptions()
    #options.add_argument('--headless')
    options.add_argument('download.default_directory={}'.format(str_def_dir_dwld))
    driver=webdriver.Chrome(executable_path=chrome_path, options=options)
    driver.implicitly_wait(10)
    return driver

def get_firefox():
    options=Options()
    
    #options.add_argument('--headless')

    profile=webdriver.FirefoxProfile()
    profile.set_preference('browser.download.folderList', 2)


    lstXLSMime=['application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
            'application/vnd.ms-excel;application/msexcel',
            'application/x-msexcel;application/x-ms-excel',
            'application/x-excel',
            'application/x-dos_ms_excel',
            'application/xls',
            'application/x-xls']

    profile.set_preference('browser.download.dir', str_def_dir_dwld)
    profile.set_preference('browser.helperApps.alwaysAsk.force', False)
    profile.set_preference('browser.helperApps.neverAsk.saveToDisk', lstXLSMime[0])

    profile.set_preference("browser.download.manager.showWhenStarting", False)
    profile.set_preference("browser.download.manager.focusWhenStarting", False)
    profile.set_preference("browser.download.manager.closeWhenDone", True)
    profile.set_preference("browser.download.manager.showAlertOnComplete", False)
    profile.set_preference("browser.download.manager.useWindow", False)
    profile.set_preference("browser.download.manager.useDownloadDir", True)
    profile.set_preference("browser.download.manager.alertOnEXEOpen", False)

    profile.set_preference("browser.download.panel.shown", False)
    profile.set_preference("services.sync.prfs.browser.download.manager.showWhenStarting", False)
    #profile.set_preference('browser.helperApps.neverAsk.openFile', 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')

    binary = FirefoxBinary('/usr/bin/firefox')
    browser = webdriver.Firefox(options=options, firefox_binary=binary)#, firefox_profile=profile)
    browser.implicitly_wait(10)
    return browser

In [4]:
def read_xlsx_files_rosstat(driver, strInn, strTargetPath=str_def_dir_dwld, bFireFox=True):
    def get_driver_spec_elem():
        if bFireFox:
            return browser.find_element(By.CLASS_NAME, 'btn.btn-primary.dropdown-toggle')
        else:
            return browser.find_element(By.CSS_SELECTOR, 'button[class="btn btn-primary dropdown-toggle"]')
    
    target_file=Path(os.path.join(strTargetPath, '{filename}.xlsx'.format(filename=strInn)))

    if not target_file.exists():

        print('Starting for INN ', strInn, end='\n')
        browser = driver
        browser.implicitly_wait(10)

        browser.get(strURL)

        button_request=browser.find_element(By.NAME, 'findOrganizations')
        txt_inn=browser.find_element(By.NAME, 'inn') 
        txt_inn.clear()
        txt_inn.send_keys(strInn)
        button_request.click()

        bt_dropdown=get_driver_spec_elem()
        
        browser.execute_script("arguments[0].click();", bt_dropdown)
        time.sleep(5)
        bt_export=browser.find_element(By.NAME, 'exportExcel')
        browser.execute_script("arguments[0].click();", bt_export)

        dwnld_file=Path(os.path.join(str_def_dir_dwld, 'Сведения о кодах и формах.xlsx'))

        tm=time.time()
    
        dtStartTime=dt.now()
        dtWork=dt(2018, 1, 1, 0, 0, 0)
        dtDelta=timedelta(seconds=5)

        while not dwnld_file.exists():
    
            dtEndTime=dt.now()
            dtWork+=dtDelta
            print('\tStarting at: {time} - working {work}'.format(time=dtStartTime.strftime('%H:%M:%S'),
                                                        work=dtWork.strftime('%H:%M:%S')), end='\r')
            time.sleep(5)
        else:
            #print(Path(os.path.join(str_def_dir_dwld, 'Сведения о кодах и формах.xlsx')))
            os.rename(dwnld_file, Path(str_def_dir_dwld, '{}.xlsx'.format(strInn)))
            
            #dwnld_file.rename(Path(str_def_dir_dwld, '{}.xlsx'.format(strInn)))
            #browser.quit()
            print('\nDone for INN ', strInn)
    else:
        print('{} already exist'.format(target_file))
        return False

In [9]:
def driver_status(driver):
    try:
        driver.execute(Command.STATUS)
        return True
    except:
        
        return False
    
driver=get_chromium() # get_firefox() #get_chromium()

for inn in lstSourceInn:
    #
    #if not driver_status(driver):
    #    driver=get_firefox() #get_chromium()
    #    time.sleep(5)
    read_xlsx_files_rosstat(driver, inn, bFireFox=False)
    
print('All done')
driver.quit()

/home/egor/Загрузки/7703585780.xlsx already exist
/home/egor/Загрузки/7736050003.xlsx already exist
/home/egor/Загрузки/7706107510.xlsx already exist
/home/egor/Загрузки/7736617998.xlsx already exist
/home/egor/Загрузки/7707572492.xlsx already exist
/home/egor/Загрузки/5504036333.xlsx already exist
/home/egor/Загрузки/2310031475.xlsx already exist
/home/egor/Загрузки/8602060555.xlsx already exist
/home/egor/Загрузки/5003021311.xlsx already exist
/home/egor/Загрузки/7705183980.xlsx already exist
/home/egor/Загрузки/7706061801.xlsx already exist
/home/egor/Загрузки/7728308080.xlsx already exist
/home/egor/Загрузки/7728029110.xlsx already exist
/home/egor/Загрузки/8608048498.xlsx already exist
/home/egor/Загрузки/7841011317.xlsx already exist
/home/egor/Загрузки/5003052454.xlsx already exist
/home/egor/Загрузки/0274051582.xlsx already exist
/home/egor/Загрузки/7840471255.xlsx already exist
/home/egor/Загрузки/4703105075.xlsx already exist
/home/egor/Загрузки/7708729065.xlsx already exist


/home/egor/Загрузки/7723011906.xlsx already exist
/home/egor/Загрузки/7838418751.xlsx already exist
/home/egor/Загрузки/3445045202.xlsx already exist
/home/egor/Загрузки/3818024303.xlsx already exist
/home/egor/Загрузки/7704731218.xlsx already exist
/home/egor/Загрузки/1655049111.xlsx already exist
/home/egor/Загрузки/7827004484.xlsx already exist
/home/egor/Загрузки/1835058718.xlsx already exist
/home/egor/Загрузки/7710419247.xlsx already exist
/home/egor/Загрузки/6606003385.xlsx already exist
/home/egor/Загрузки/6317019121.xlsx already exist
/home/egor/Загрузки/6167055801.xlsx already exist
/home/egor/Загрузки/4027091179.xlsx already exist
/home/egor/Загрузки/7703755618.xlsx already exist
/home/egor/Загрузки/8904047896.xlsx already exist
/home/egor/Загрузки/7731530768.xlsx already exist
/home/egor/Загрузки/7718852163.xlsx already exist
/home/egor/Загрузки/4312138386.xlsx already exist
/home/egor/Загрузки/8904037947.xlsx already exist
/home/egor/Загрузки/5047149534.xlsx already exist


KeyboardInterrupt: 

In [6]:
driver.quit()